In [ ]:
import pandas as pd
from sklearn.svm import SVC

# Load data

In [ ]:
data1 = download_data1()

In [ ]:
data2 = download_data2()

# Combine data

In [ ]:
data = pd.concat([data1, data2])

# Preprocess data

In [ ]:
data = preprocess(data)

# Train model

In [ ]:
model = SVC()

In [ ]:
model.fit(data[:,:-1], data['target'])

In [ ]:
class TaskGetData1(d6tflow.tasks.TaskPqPandas):
    
    def run():
        data = download_data1()
        self.save(data)

In [ ]:
class TaskGetData2(d6tflow.tasks.TaskPqPandas):
    
    def run():
        data = download_data2()
        self.save(data)

In [ ]:
class TaskCombineData(d6tflow.tasks.TaskPqPandas):
    do_preprocess = luigi.BoolParameter(default=True) # parameter for preprocessing

    def requires(self):
        return TaskGetData1(), TaskGetData2() # define dependency
    
    def run(self):
        data1, data2 = self.loadInputs() # load input data
        data = pd.concat([data1, data2])
        
        if do_preprocess:
            data = preprocess(data)
        
        self.save(data) # save output data

In [ ]:
@inherits(TaskCombineData) # pass parameters up to parent
class TaskTrain():
    
    def requires(self):
        return self.clone_parent()
    
    def run(self):
        data = self.input().load()
        model = SVC()
        model.fit(data[:,:-1], data['target'])
        self.save(model)

In [ ]:
d6tflow.run(TaskTrain(do_preprocess=True))